<a href="https://colab.research.google.com/github/vchandraiitk/datascience/blob/main/M7_NB_MiniProject_2_Simple_Analytics_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Simple Analytics using Pyspark

## Problem Statement

Perform simple analytics with Pyspark on the real estate valuation dataset and predict the house price per unit area

## Learning Objectives

At the end of the mini-project, you will be able to :

* analyze the data using pyspark
* derive the insights and visualize the data
* implement linear regression and evaluate using pyspark

### Dataset

The dataset chosen for this mini-project is **Real Estate Valuation dataset**. The data was collected from the historical market of real estate within Sindian District of New Taipei City, the timespan across 2012 August to 2013 July. In the dataset, the response variable (house price per unit area) is calculated in a local unit which is approximately $10000 New Taipei Dollar per 3.3 squared meters. For the collection of regressor data, the transaction dates are transformed into a format such that 2013.250 = 2013 March, 2013.500 = 2013 June etc. The house age was collected in years and the distance to MRT stations is measured in meters.

**Reference:**
The original owner of this Real Estate Valuation dataset is professor I-Cheng Yeh from TamKang University (Department of Civil Engineering). Prof. Yeh donated this dataset to UCI machine learning repository on 18th August 2018. The dataset can be accessed at https://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set#[1].

## Grading = 10 Points

#### Install Pyspark

In [1]:
#@title Install pyspark and Download the data
!pip -qq install pyspark
!pip -qq install handyspark
!pip install numpy==1.23.0
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/RealEstate.csv
print("Packages installed and dataset downloaded successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.0 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.0 which is incompatible.
bigframes 1.30.0 requires numpy>=1.24.0, but you have numpy 1.23.0 which is incompatible.
chex 0.1.88 requires numpy>=1.24.1, but you have numpy 1.23.0 which is incompatible.
ibis-framework 9.2.0 requires numpy<3,>=1.23.2, but you have numpy 1.23.0 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.0 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but yo

Packages installed and dataset downloaded successfully!


#### Import required packages

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import MinMaxScaler
from handyspark import *
import seaborn as sns
from matplotlib import pyplot as plt

### Data Loading (1 point)

#### Start a Spark Session

Spark session is a combined entry point of a Spark application, which came into implementation from Spark 2.0. It provides a way to interact with various spark’s functionality with a lesser number of constructs.

In [2]:
spark = SparkSession.builder.appName('RealEstate').getOrCreate()

#### Load the data and infer the schema

To load the dataset use the `read.csv` with `inferSchema` and `header` as parameters.

In [3]:
csv_path = "/content/RealEstate.csv"
# YOUR CODE HERE
dataset = spark.read.csv(csv_path, inferSchema=True, header=True)

#### Fetch the data using handyspark

* Create a HandyFrame using handyspark `toHandy()` function

* using an instance of `cols` from your HandyFrame, you can retrieve values for given columns in the top N rows

Hint: [toHandy()](https://dvgodoy.github.io/handyspark/includeme.html)

In [4]:
# YOUR CODE HERE
handyFrame = dataset.toHandy()
handyFrame.show(5)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+
| No|X1 transaction date|X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|X5 latitude|X6 longitude|Y house price of unit area|
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+
|  1|           2012.917|        32.0|                              84.87882|                             10|   24.98298|   121.54024|                      37.9|
|  2|           2012.917|        19.5|                              306.5947|                              9|   24.98034|   121.53951|                      42.2|
|  3|           2013.583|        13.3|                              561.9845|                              5|   24.98746|   121.54391|                      47.3|
|  4|             2013.5|   

### Deriving the insights (2 points)

#### show the no. of records per month

Hint: Apply [groupby](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.GroupedData.agg.html) on transaction date and count the records using aggregation `agg()`

In [5]:
# YOUR CODE HERE
dataset.groupBy("X1 transaction date").agg(count("X1 transaction date")).show()

+-------------------+--------------------------+
|X1 transaction date|count(X1 transaction date)|
+-------------------+--------------------------+
|            2012.75|                        27|
|           2013.333|                        29|
|           2013.083|                        46|
|           2012.833|                        31|
|             2013.5|                        47|
|           2013.583|                        23|
|           2013.417|                        58|
|           2013.167|                        25|
|            2013.25|                        32|
|             2013.0|                        28|
|           2012.917|                        38|
|           2012.667|                        30|
+-------------------+--------------------------+



#### how much is the increase in the average house price in 2012 to 2013

Hint: Apply [filter](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.filter.html) on the transaction date and aggregate the house price using mean

In [6]:
dataset.printSchema()

root
 |-- No: integer (nullable = true)
 |-- X1 transaction date: double (nullable = true)
 |-- X2 house age: double (nullable = true)
 |-- X3 distance to the nearest MRT station: double (nullable = true)
 |-- X4 number of convenience stores: integer (nullable = true)
 |-- X5 latitude: double (nullable = true)
 |-- X6 longitude: double (nullable = true)
 |-- Y house price of unit area: double (nullable = true)



In [7]:
# YOUR CODE HERE
from pyspark.sql.types import IntegerType # Import IntegerType

avg_2012 = dataset.filter(dataset["X1 transaction date"].cast(IntegerType()) == 2012).agg(avg("Y house price of unit area")).collect()
avg_2013 = dataset.filter(dataset["X1 transaction date"].cast(IntegerType()) == 2013).agg(avg("Y house price of unit area")).collect()
avg_2013[0][0] - avg_2012[0][0]

2.4084325396825506

#### Find the count of houses with no convenience store and show the top 10 records

In [8]:
# YOUR CODE HERE
dataset.filter(dataset["X4 number of convenience stores"] == 0).count()

67

#### Compare the maximum price of houses *with convenient store* and *without convenience store*

In [22]:
# YOUR CODE HERE
dataset.groupBy("X4 number of convenience stores").agg(max("Y house price of unit area")).show()
dataset.groupBy("X4 number of convenience stores").agg(min("Y house price of unit area")).show()

+-------------------------------+-------------------------------+
|X4 number of convenience stores|max(Y house price of unit area)|
+-------------------------------+-------------------------------+
|                              1|                          117.5|
|                              6|                           73.6|
|                              3|                           61.5|
|                              5|                           60.7|
|                              9|                           78.3|
|                              4|                           62.9|
|                              8|                           67.7|
|                              7|                           62.1|
|                             10|                           61.9|
|                              2|                           50.5|
|                              0|                           55.3|
+-------------------------------+-------------------------------+

+--------

In [27]:
from pyspark.sql.functions import col
dataset.filter(col("X4 number of convenience stores") > 0).agg(max("Y house price of unit area").alias("Max with Convenience Stores")).show()
dataset.filter(col("X4 number of convenience stores") == 0).agg(max("Y house price of unit area").alias("Max w/o Convenience Stores")).show()

+---------------------------+
|Max with Convenience Stores|
+---------------------------+
|                      117.5|
+---------------------------+

+--------------------------+
|Max w/o Convenience Stores|
+--------------------------+
|                      55.3|
+--------------------------+



#### Decode the transaction date

* create a year column separately by removing the decimal places from transaction date column

    Hint: `withColumn()`

* create a month column separately based on the decimal places from the transaction date column

 **Hints:** multiply decimal place number with 12 and apply round off, perform below steps
     
     - use `udf()` from pyspark
     - subtracting `transaction date` from `int(transaction date)` will give the decimal place number
     - multiply the result with 12 and roundoff

To know more about udf(), click [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.udf.html)

In [30]:
# YOUR CODE HERE
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
dataset = dataset.withColumn("Year", col("X1 transaction date").cast(IntegerType()))

In [31]:
dataset.show(5)

+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+----+
| No|X1 transaction date|X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|X5 latitude|X6 longitude|Y house price of unit area|Year|
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+----+
|  1|           2012.917|        32.0|                              84.87882|                             10|   24.98298|   121.54024|                      37.9|2012|
|  2|           2012.917|        19.5|                              306.5947|                              9|   24.98034|   121.53951|                      42.2|2012|
|  3|           2013.583|        13.3|                              561.9845|                              5|   24.98746|   121.54391|                      47.3|2013

In [60]:
from pyspark.sql.functions import udf, round, floor
from pyspark.sql.types import IntegerType

def extract_month(date_val):
    # Use PySpark's floor and cast for Column operations
    return round((date_val - floor(date_val)) * 12).cast(IntegerType())

extract_month_udf = udf(extract_month, IntegerType())
dataset = dataset.withColumn("Month", extract_month_udf(col("X1 transaction date")))
dataset.show(5)

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "<ipython-input-60-a4b50bd1d61e>", line 6, in extract_month
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 174, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 1591, in floor
    return _invoke_function_over_columns("floor", col)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in _invoke_function_over_columns
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in <genexpr>
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/usr/local/lib/python3.10/

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "<ipython-input-57-3d4ec2c77a85>", line 5, in extract_month
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 174, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 1591, in floor
    return _invoke_function_over_columns("floor", col)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in _invoke_function_over_columns
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/functions.py", line 105, in <genexpr>
    return _invoke_function(name, *(_to_java_column(col) for col in cols))
  File "/usr/local/lib/python3.10/

In [ ]:
def extract_month(date_val):
    return round((date_val - int(date_val)) * 12)

extract_month_udf = udf(extract_month, IntegerType())
dataset = dataset.withColumn("Month", extract_month_udf(col("X1 transaction date")))

### Data Visualization (2 points)

#### Select the continuous variables from the data and visualize using histogram

In [ ]:
# YOUR CODE HERE

#### Visualize the transaction date using the countplot

In [ ]:
# YOUR CODE HERE

#### visualize number of convenience stores with appropriate plot.

In [ ]:
# YOUR CODE HERE

#### visualize the geographical distribution of the house prices of unit area

With:

* x-axis = X6 longitude
* y-axis = X5 latitude
* datapoints = Y house price of unit area
* parameters including size `s` and color map `cmap`

In [ ]:
# YOUR CODE HERE

### Feature Scaling (1 point)

#### Identify the outliers

Use the pairplot or boxplot to identify the outliers

   **Hint:** `sns.pairplot`

In [ ]:
# YOUR CODE HERE

#### Correlation analysis

create the correlation matrix of all the columns and visualize using the heatmap

In [ ]:
# YOUR CODE HERE

#### Normalization or standardization

select the applicable features using vector assembler and apply scaling using `MinMaxScaler` from pyspark

Hint: [MinMaxScaler](https://spark.apache.org/docs/latest/ml-features#minmaxscaler)

In [ ]:
# YOUR CODE HERE

### Feature Engineering (2 points)

#### Assemble the features

concatenate all the features into a single vector which can be further passed to the ML algorithm.

**Hint:** `VectorAssembler()`

In [ ]:
# YOUR CODE HERE

#### Feature selection and splitting the data

In [ ]:
# YOUR CODE HERE

### Train and Evaluate the model (2 points)

Using LinearRegression from `pyspark.ml` fit the data and find the coefficients and intercept

In [ ]:
regressor = LinearRegression(featuresCol='features', labelCol='label')

In [ ]:
# YOUR CODE HERE

Get the predictions and show the table along with actual data and predicted data

In [ ]:
# YOUR CODE HERE

Calculate the RMSE and $R^2$ of the model

In [ ]:
# YOUR CODE HERE